<a href="https://colab.research.google.com/github/maithoi/thesis/blob/main/convert_label_to_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries & dependences 


In [ ]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

# Connect with Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 0.Thesis
 1.Career
 22feb-20mar.gsheet
'9-DCMH - Mang may tinh - 405015.gdoc'
'Beat Task1.gdoc'
'Beautiful words.gsheet'
'Big Pictures.gjam'
 BME-Assignment.gdoc
'Cal Newport - So good they can'\''t ignore you_ why skills trump passion in the quest for work you love-Business Plus (2012).mobi'
 Careeer.gdoc
 CAREERS
'Chapter10 - Email.gslides'
'Chapter11-Network Managment-3ed.gslides'
'Chapter1 - Data link Layer (1).gslides'
'Chapter1 - Data link Layer.gslides'
 Chapter1-Introduction.gslides
'Chapter2- Mac Protocol.gslides'
'Chapter3 - Spanning Tree.gslides'
'Chapter4 - Internet Protocol.gslides'
'Chapter5 - Routing.gslides'
'Chapter6 - Transport Layer.gslides'
'Chapter7 - DNS.gslides'
'Chapter8 - FTP (1).gslides'
'Chapter8 - FTP (2).gslides'
'Chapter8 - FTP (3).gslides'
'Chapter8 - FTP (4).gslides'
'Chapter8 - FTP.gslides'
'Chapter9 - HTTP.gslides'
'Chrome OS Cloud backup'
'Coding Interview - Practice where It hurts.gdoc'
'Colab Notebooks'
 cominghome2020.pd

# Download Yolov5 Model 

In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 3390, done.
remote: Total 3390 (delta 0), reused 0 (delta 0), pack-reused 3390
Receiving objects: 100% (3390/3390), 7.02 MiB | 10.93 MiB/s, done.
Resolving deltas: 100% (2274/2274), done.
/content/yolov5


In [ ]:
!mkdir -p Dataset/FaceMask/Images
!mkdir -p Dataset/FaceMask/Labels

In [ ]:
!pwd

/content/yolov5


# Copy image files 

In [ ]:
print(os.listdir("/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection /images"))

['maksssksksss0.png', 'maksssksksss1.png', 'maksssksksss128.png', 'maksssksksss110.png', 'maksssksksss111.png', 'maksssksksss101.png', 'maksssksksss135.png', 'maksssksksss13.png', 'maksssksksss103.png', 'maksssksksss104.png', 'maksssksksss107.png', 'maksssksksss119.png', 'maksssksksss133.png', 'maksssksksss137.png', 'maksssksksss11.png', 'maksssksksss127.png', 'maksssksksss112.png', 'maksssksksss132.png', 'maksssksksss126.png', 'maksssksksss120.png', 'maksssksksss122.png', 'maksssksksss134.png', 'maksssksksss109.png', 'maksssksksss105.png', 'maksssksksss123.png', 'maksssksksss12.png', 'maksssksksss138.png', 'maksssksksss131.png', 'maksssksksss136.png', 'maksssksksss124.png', 'maksssksksss10.png', 'maksssksksss100.png', 'maksssksksss118.png', 'maksssksksss117.png', 'maksssksksss102.png', 'maksssksksss129.png', 'maksssksksss106.png', 'maksssksksss130.png', 'maksssksksss139.png', 'maksssksksss113.png', 'maksssksksss125.png', 'maksssksksss115.png', 'maksssksksss116.png', 'maksssksksss121.p

In [ ]:
# upload files into Dataset/FaceMask/Images 
from google.colab import files
uploaded = files.upload()

In [ ]:
!cp -rf /content/gdrive/MyDrive/face_mask_detection /images Dataset/FaceMask/Images

cp: cannot stat '/content/gdrive/MyDrive/face_mask_detection': No such file or directory
cp: cannot stat '/images': No such file or directory


# Create destinations images path & labels path 

In [ ]:
!mkdir -p Dataset/images Dataset/labels

# Create Dataset 

In [ ]:
# FILE_ROOT = "/kaggle/input/face-mask-detection /"
# IMAGE_PATH = FILE_ROOT + "images"  
# ANNOTATIONS_PATH = FILE_ROOT + "annotations"

# DATA_ROOT = "Dataset/"
# LABELS_ROOT = DATA_ROOT + "FaceMask/Labels"
# IMAGES_ROOT = DATA_ROOT + "FaceMask/Images"  

# DEST_IMAGES_PATH = "images"
# DEST_LABELS_PATH = "labels" 

In [ ]:
FILE_ROOT = "/content/gdrive/MyDrive/face_mask_detection /"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "Dataset/"
LABELS_ROOT = DATA_ROOT + "FaceMask/Labels"
IMAGES_ROOT = DATA_ROOT + "FaceMask/Images"  

DEST_IMAGES_PATH = "images"
DEST_LABELS_PATH = "labels" 

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

### convert annotations (from COCO .xml to YOLO format .txt)

In [ ]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = cord_converter(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
print(os.listdir("/content/gdrive/MyDrive/face_mask_detection /annotations"))

['maksssksksss101.xml', 'maksssksksss105.xml', 'maksssksksss109.xml', 'maksssksksss1.xml', 'maksssksksss107.xml', 'maksssksksss100.xml', 'maksssksksss11.xml', 'maksssksksss108.xml', 'maksssksksss10.xml', 'maksssksksss0.xml', 'maksssksksss103.xml', 'maksssksksss106.xml', 'maksssksksss104.xml', 'maksssksksss110.xml', 'maksssksksss102.xml', 'maksssksksss532.xml', 'maksssksksss189.xml', 'maksssksksss352.xml', 'maksssksksss29.xml', 'maksssksksss449.xml', 'maksssksksss842.xml', 'maksssksksss208.xml', 'maksssksksss649.xml', 'maksssksksss52.xml', 'maksssksksss56.xml', 'maksssksksss325.xml', 'maksssksksss421.xml', 'maksssksksss134.xml', 'maksssksksss397.xml', 'maksssksksss758.xml', 'maksssksksss156.xml', 'maksssksksss83.xml', 'maksssksksss824.xml', 'maksssksksss597.xml', 'maksssksksss505.xml', 'maksssksksss627.xml', 'maksssksksss162.xml', 'maksssksksss834.xml', 'maksssksksss424.xml', 'maksssksksss411.xml', 'maksssksksss174.xml', 'maksssksksss514.xml', 'maksssksksss547.xml', 'maksssksksss251.xml

In [ ]:
print(os.listdir("/content/gdrive/MyDrive/face_mask_detection /images"))

['maksssksksss0.png', 'maksssksksss1.png', 'maksssksksss128.png', 'maksssksksss110.png', 'maksssksksss111.png', 'maksssksksss101.png', 'maksssksksss135.png', 'maksssksksss13.png', 'maksssksksss103.png', 'maksssksksss104.png', 'maksssksksss107.png', 'maksssksksss119.png', 'maksssksksss133.png', 'maksssksksss137.png', 'maksssksksss11.png', 'maksssksksss127.png', 'maksssksksss112.png', 'maksssksksss132.png', 'maksssksksss126.png', 'maksssksksss120.png', 'maksssksksss122.png', 'maksssksksss134.png', 'maksssksksss109.png', 'maksssksksss105.png', 'maksssksksss123.png', 'maksssksksss12.png', 'maksssksksss138.png', 'maksssksksss131.png', 'maksssksksss136.png', 'maksssksksss124.png', 'maksssksksss10.png', 'maksssksksss100.png', 'maksssksksss118.png', 'maksssksksss117.png', 'maksssksksss102.png', 'maksssksksss129.png', 'maksssksksss106.png', 'maksssksksss130.png', 'maksssksksss139.png', 'maksssksksss113.png', 'maksssksksss125.png', 'maksssksksss115.png', 'maksssksksss116.png', 'maksssksksss121.p

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

file name:  maksssksksss101.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss101.xml
Dataset/FaceMask/Labels/maksssksksss101.txt
file name:  maksssksksss105.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss105.xml
Dataset/FaceMask/Labels/maksssksksss105.txt
file name:  maksssksksss109.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss109.xml
Dataset/FaceMask/Labels/maksssksksss109.txt
file name:  maksssksksss1.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss1.xml
Dataset/FaceMask/Labels/maksssksksss1.txt
file name:  maksssksksss107.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss107.xml
Dataset/FaceMask/Labels/maksssksksss107.txt
file name:  maksssksksss100.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/maksssksksss100.xml
Dataset/FaceMask/Labels/maksssksksss100.txt
file name:  maksssksksss11.xml
/content/gdrive/MyDrive/face_mask_detection /annotations/

# Download converted YOLO TXT

In [ ]:
# !zip -r /content/file.zip /content/Folder_To_Zip
!zip -r /content/labels.zip /content/yolov5/Dataset/FaceMask/Labels

  adding: content/yolov5/Dataset/FaceMask/Labels/ (stored 0%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss306.txt (deflated 26%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss493.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss108.txt (deflated 57%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss470.txt (deflated 55%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss192.txt (deflated 26%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss8.txt (deflated 27%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss748.txt (deflated 29%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss460.txt (deflated 58%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss276.txt (deflated 42%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss540.txt (deflated 26%)
  adding: content/yolov5/Dataset/FaceMask/Labels/maksssksksss262.txt (deflated 28%)
  adding: conten

# Split images dataset 

In [ ]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('Dataset/FaceMask/Images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=7)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=8)

print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

ValueError: ignored